In [1]:
import sys
import os
import rasterio

module_path = os.path.abspath(os.path.join('/workspace/Transformer_OD_TPU'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "6" 

In [3]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

In [4]:
import argparse
import datetime
import json
import random
import time
from pathlib import Path
from tqdm import tqdm

import contextlib
import io

import numpy as np
import torch
from torch.utils.data import DataLoader, DistributedSampler
torch.set_printoptions(sci_mode=False)

import datasets
import util.misc as utils
from datasets import build_dataset, get_coco_api_from_dataset
from engine import evaluate, train_one_epoch
from models import build_model

In [5]:
import main

In [36]:
crop = 256
dataset = 'transfer_data'

In [37]:
experiment = f'detr_output_{crop}_binary_skip'
if dataset == 'initial_data':
    mean = 6.6374
    std = 10.184
elif dataset == 'transfer_data':
    mean = 0.7294
    std = 9.3929
output_dir = os.path.join('/workspace/',experiment)

sys.argv =  ['main.py', '--num_classes', '2', '--coco_path', f'/workspace/processed_data_v7/{dataset}', '--num_workers', '0', '--resume', f'{output_dir}/best_checkpoint.pth', '--batch_size', '1', '--crop', f'{crop}']

In [38]:
parser = main.get_args_parser()
args = parser.parse_args()

In [39]:
dataset_test = build_dataset(image_set='test', args=args)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [40]:
sampler_test = torch.utils.data.SequentialSampler(dataset_test)
data_loader_test = DataLoader(dataset_test, 1, sampler=sampler_test,
                            drop_last=False, collate_fn=utils.collate_fn, num_workers=args.num_workers)

In [41]:
device = torch.device(args.device)
model, criterion, postprocessors = build_model(args)
model = model.to(device).eval()

In [42]:
checkpoint = torch.load(args.resume, map_location='cpu')
model.load_state_dict(checkpoint['model'])

<All keys matched successfully>

In [43]:
from datasets.coco_eval import CocoEvaluator
base_ds = get_coco_api_from_dataset(dataset_test)

In [44]:
%%capture
test_stats, coco_evaluator = evaluate(
    model, criterion, postprocessors, data_loader_test, base_ds, device, '/workspace'
)

In [45]:
buffer = io.StringIO()

# use contextlib.redirect_stdout to redirect the output to the buffer
with contextlib.redirect_stdout(buffer):
    coco_evaluator.coco_eval['bbox'].summarize()
output = buffer.getvalue()

# write the output to a file
with open(os.path.join(output_dir, f'coco_eval_output_{dataset}.txt'), 'w') as file:
    file.write(output)